In [1]:
%reset -f

from ipywidgets import *
import ipympl
import matplotlib.pyplot as plt

import sys
import proteus
## Required imports
from proteus.iproteus import * 
from proteus import default_n, default_s, default_so, Comm, Context
from proteus.mprans import SpatialTools as st
from petsc4py import PETSc
from threading import Thread


reload(Context)
import tempFlume as plant
reload(plant)
import tempFlume_so as plant_so
reload(plant_so)
#


import numpy as np
from IPython.display import clear_output, display,FileLink,FileLinks

from tables import  openFile
from collections import  namedtuple



#import PostProcessing 
#reload(PostProcessing)
import helpers
reload(helpers)
import utilities
reload(utilities)
import PostProcessing
reload(PostProcessing)

from IPython.core.display import HTML
import webbrowser

global simulation_thread, ns

plt.close("all")

def on_value_change(change):
    clear_output()
    
    Length=tankLength.value
    Height=tankHeight.value
    x_en=CB_enableX.value
    y_en=CB_enableY.value
    theta_en=CB_enableTheta.value
    abs_en=CB_enableAbsortion.value

    #Change Context
    plant.opts.water_level=waterZ.value
    plant.opts.inflow_velocity=sld_flowvelocity.value
    plant.opts.outflow_velocity=sld_flowvelocity.value
    plant.opts.absorption=abs_en
    plant.opts.caisson=CB_addCaisson.value
    plant.opts.caisson_scale=sld_caissonScale.value
    plant.opts.rotation_angle=sld_Angle.value
    plant.opts.waves=CB_addWaves.value
    plant.opts.waveHeight=sld_WavesHeight.value/1000
    plant.opts.caisson_Yoffset=sld_CaissonOffset.value
    plant.opts.tank_dim=(Length,Height)
    plant.opts.free_x=(x_en, y_en, 0.0)
    plant.opts.free_r=(0.0, 0.0, theta_en)
    plant.opts.movingDomain=any([x_en,y_en,theta_en]) #Useless unless plant_so is modified
    plant.Update_Model()
    
    Context.setFromModule(plant,mutable=True)
    ct = Context.get()
    plant_so.ct=ct
    
    #Plot new tank
    helpers.f=plt.figure(num=1)
    helpers.plot_domain()
    
    
# Create Tank Dimension Widgets   
#Tank
tankLength=FloatSlider(min=0.0, max=10.0, step=0.05, continuous_update=False,value=plant.tank_dim[0],
                            description='Tank Length:')
tankHeight=FloatSlider(min=0.1, max=3.0, step=0.05, continuous_update=False,value=plant.tank_dim[1],
                            description='Tank Height:')
#Caisson
CB_addCaisson=Checkbox(value=True, description='Add Caisson', disabled=False)
sld_caissonScale=IntSlider(min=1, max=30, step=1, continuous_update=False,value=plant.opts.caisson_scale,
                            description='Scale:')
sld_Angle=IntSlider(min=-45, max=45, step=5, continuous_update=False,value=plant.opts.rotation_angle,
                            description='Rotation (deg):')
sld_CaissonOffset=FloatSlider(min=-1.0, max=1.0, step=0.00001, continuous_update=False,
                              value=plant.opts.caisson_Yoffset, description='Offset')
CB_enableTheta=Checkbox(value=False, description='Theta')
CB_enableX=Checkbox(value=False, description='X')
CB_enableY=Checkbox(value=False, description='Y')
#Water
waterZ=FloatSlider(min=0.0, max=3.0, step=0.05, continuous_update=False,value=plant.opts.water_level,
                            description='Water Level:')
#waterX=FloatSlider(min=0.1, max=10, step=0.05, continuous_update=False,value=plant.opts.water_width_over_obst,
#                      description='Water width over obstacle:')     
sld_flowvelocity=FloatSlider(min=0., max=0.5, step=0.01, continuous_update=False,value=plant.opts.inflow_velocity,
                            description='Flow Velocity:')                      
CB_addWaves=Checkbox(value=plant.opts.waves, description='Enable Waves', disabled=False)
sld_WavesHeight=IntSlider(min=0, max=int(1000*plant.tank_dim[1]/10),
                            step=1, continuous_update=False,value=1000*plant.opts.waveHeight,
                            description='Wave Height (mm):')  
CB_enableAbsortion=Checkbox(value=True, description='Add Absortion Zones')


tankBox  = VBox([tankLength, tankHeight])
obsBox   = VBox([CB_addCaisson, sld_caissonScale, sld_Angle,
                 sld_CaissonOffset, CB_enableTheta, CB_enableX, CB_enableY])
waterBox = VBox([waterZ, sld_flowvelocity,CB_addWaves,sld_WavesHeight,CB_enableAbsortion])

page1Controls=HBox([tankBox,obsBox,waterBox])

for children_1 in enumerate(page1Controls.children):
    for children_2 in enumerate(children_1[1].children):
        print children_2[1].observe(on_value_change, names='value')


f1 = plt.figure()
plt.plot([])

page1=VBox([page1Controls,f1.canvas])
on_value_change(True)

# Link widget values with the current tank dimension values
#mylink1 = jslink((tankLength, 'value'), (caissonWidth, 'max'))
#mylink2 = jslink((tankHeight, 'value'), (caissonHeight, 'max'))
mylink3 = jslink((tankHeight, 'value'), (waterZ, 'max'))
#mylink3 = jslink((tankHeight, 'value'), (sld_WavesHeight, 'max'))
#mylink4 = jslink((tankHeight, 'value'), (waterX, 'max'))

#============================================================================================================================
#Physics and Numerics
#Load the modules the define the equations to be solved and the numerical methods to use.
#============================================================================================================================


#Prepare Widgets
txb_refine=IntText(value=plant.opts.refinement,step=1,
                        description='Refinement')
txb_he=FloatText(value=plant.opts.he, step=1e-15,
                    description='he', disabled=False)
txb_he_caisson=FloatText(value=plant.opts.he_caisson, step=1e-15,
                    description='he_caisson', disabled=False)
showTriangles=widgets.Checkbox(value=True, description='Show Triangles', disabled=False)
txb_dtInit=FloatText(value=plant.opts.dt_init,step=.001,
                        description='dt initial',disabled=False)
txb_dtFixed=FloatText(value=plant.opts.dt_fixed,step=.001,
                       description='dt')
txb_tf=FloatText(value=plant.opts.T,step=.001,
                        description='Simulation Time')
showAirWaterSpeed=widgets.Checkbox(value=False, description='Show Air/Water Speed', disabled=False)
btn_parameters = widgets.Button(description="Run Simulation")

#Button simulate code
def on_button_clicked(b):
    global simulation_thread, ns
    clear_output()
    print("Computing Parameters...")
    !rm mesh.*
    #plant.opts.refinement = txb_refine.value
    plant.opts.dt_init = txb_dtInit.value
    plant.opts.dt_fixed = txb_dtFixed.value
    plant.opts.T = txb_tf.value
    plant.opts.he = txb_he.value
    plant.opts.he_caisson = txb_he_caisson.value
    plant.Update_Model()
    #plant.triangleOptions="VApq30Dena%8.8f" % ((txb_he.value**2)/2.0,)
    Context.setFromModule(plant,mutable=True)
    ct = Context.get()
    plant_so.ct=ct
    
    #attrs = vars(ct.domain.MeshOptions)
    #for item in attrs.items():
    #    print "attr: %s \n value:%s" % item 

    so = plant_so
    so.tnList = [0.0,plant.dt_init]+[i*plant.dt_fixed for i in range(1,plant.nDTout+1)]
    info = open("TimeList.txt","w")
    for time in so.tnList:
        info.write(str(time)+"\n")
    info.close()
    plant_so.tnList=so.tnList
    pList=[]
    nList=[]
    so.sList=[]
    OptDB = PETSc.Options()
    for (p,n) in so.pnList:
        so.sList.append(default_s)
        pList.append(__import__(p))
        reload(pList[-1])
        nList.append(__import__(n))
        reload(nList[-1])
        pList[-1].name = p
        nList[-1].multilevelLinearSolver = default_n.KSP_petsc4py
        nList[-1].levelLinearSolver = default_n.KSP_petsc4py
        OptDB.setValue(nList[-1].linear_solver_options_prefix+"ksp_type", "preonly")
        OptDB.setValue(nList[-1].linear_solver_options_prefix+"pc_type", "lu")
        OptDB.setValue(nList[-1].linear_solver_options_prefix+"pc_factor_mat_solver_package","superlu_dist")
    opts.save_dof = True
    opts.dataDir='.'
    opts.probDir='.'
    opts.logLevel=7
    opts.verbose=True
    opts.viewMesh=True
    
    #-----------------------------------------------------------------------
    ## Numerical Solution Object
    #    Now we create an the numerical wavetank object and set it up to run in a thread on each engine.
    #-----------------------------------------------------------------------
    #!rm *.h5
    #!rm *.xmf
    print("Creating Numerical Solution Object...")
    import time
    t1=time.time()
    ns = NumericalSolution.NS_base(so, pList, nList, so.sList, opts)
    # Create a thread wrapper for the simulation.  The target must be an argument-less
    # function so we wrap the call to `calculateSolution` in a simple lambda:
    print("Creating Simulation Thread...")
    simulation_thread = Thread(target = lambda : ns.calculateSolution('run1'))
    print("Starting Thread and Simulation Monitor")
    simulation_thread.start()
    f2.clear()
    plt.plot([])
    f2.canvas.draw()

    utilities.simulation_thread=simulation_thread
    utilities.ns=ns
    utilities.plt=plt
    utilities.f=plt.figure(num=2)
    utilities.plant=plant
    utilities.showTriangles=showTriangles.value
    utilities.showAir=showAirWaterSpeed.value
    utilities.monitor_simulation(refresh=5.0)
    #clear_output()
    print("Complete in:")
    t2=time.time()
    print(t2-t1)

    
f2=plt.figure()
f2.clear()
plt.plot([])
f2.canvas.draw()
    
btn_parameters.on_click(on_button_clicked)
Sim_Controls=VBox([txb_tf,
                   txb_dtFixed,
                   txb_dtInit,
                   txb_he,
                   txb_he_caisson,
                   #txb_refine,
                   showTriangles,
                   showAirWaterSpeed,
                   btn_parameters])
page3=HBox([Sim_Controls,f2.canvas])



#============================================================================================================================
# Create Post-Processing
#============================================================================================================================
f3=plt.figure()
f3.clear()
plt.plot([])
f3.canvas.draw()
 

btn_createImages=Button(description='Create Images')

def on_btn_createImages_clicked(b):
    clear_output()
    print("Creating Images ...")
    f3.clear()
    !rm ./PostProcessing/phi*png
    PostProcessing.plant=plant
    PostProcessing.plt=plt
    PostProcessing.f=plt.figure(num=3)
    PostProcessing.plant_so=plant_so
    name="tempFlume_motion"
    PostProcessing.Create_Images('tempFlume_motion')
    clear_output()
    f3.canvas.draw()
    print('Images Created')

btn_createImages.on_click(on_btn_createImages_clicked)
btn_createVideo=Button(description='Create Video',)
txb_rate=IntText(value=int(round(1/txb_dtFixed.value)),step=1,
                        description='Frame Rate')
txb_Name=Text(value='DefaultVideoName',
                placeholder='Type something',
                description='Video Name:',
                disabled=False
                )
def on_btn_createVideo_clicked(b):
    clear_output()
    print("Creating Video...")
    rate=txb_rate.value
    name=txb_Name.value
    name="./PostProcessing/"+name
    !rm -f {name}.mp4; LD_LIBRARY_PATH='' avconv -r {rate} -i  ./PostProcessing/phi%4d.png -vcodec libx264 {name}.mp4 -loglevel quiet
    #PostProcessing.Create_Video(name)
    clear_output()
    print("Click link to see video:")
    display(FileLink(name[2:]+".mp4"))
    
btn_createVideo.on_click(on_btn_createVideo_clicked)

page4=HBox([VBox([btn_createImages,btn_createVideo,txb_Name,txb_rate]), f3.canvas])


#============================================================================================================================
# Widget Display
#============================================================================================================================

# Display Widgets in Tabs
lst=['Bathmetry and SDF','Simulation','Post Processing']
children = [page1,page3,page4]
tab = widgets.Tab(children=children)
[tab.set_title(num, name) for num, name in enumerate(lst)]
display(tab)


Computing Parameters...
Creating Numerical Solution Object...
Creating Simulation Thread...
Starting Thread and Simulation Monitor
u_max=0.274, v_max=0.000, Vmax=0.274, cfl=0.000, dt=1.00000
Monitored for: 0:00:04.234683. at t= 1.00000e-04


IndexError: list index out of range

In [ ]:

import pandas as pd    
import math
import numpy as np
import ipympl
import matplotlib.pyplot as plt
from IPython.display import clear_output, display
from ipywidgets import *
xl = pd.ExcelFile("Ribbon Bridge Section Line Segment.xlsx")
df = xl.parse(0)
xp=np.asarray(df['x'].tolist())
yp=np.asarray(df['y'].tolist())
# Offset to (0,0)
xp-=0.5*(max(xp)+min(xp))
yp-=0.5*(max(yp)+min(yp))
scale=1./25.
# Scale to experimental model 
xp=xp*scale
yp=yp*scale


he_min=0.01

#Offset top corners for interpolation
xp[0]+=he_min/100
xp[-1]-=he_min/100

xd =np.diff(xp)
yd = np.diff(yp)
dist = np.sqrt(xd**2+yd**2)
u = np.cumsum(dist)
u = np.hstack([[0],u])

t = np.linspace(0,u.max(),int(u.max()/he_min))
xBase = np.interp(t, u, xp)
yBase = np.interp(t, u, yp)

xTop= np.linspace(xp.min()+he_min,xp.max(),int((xp.max()-xp.min())/he_min), endpoint=False)
yTop=np.full(len(xTop),yp[-1])
x=np.hstack([xBase,xTop])
y=np.hstack([yBase,yTop])
print x
print y

f4=plt.figure()
f4.clear()
plt.plot(xp,yp,'-o')
plt.plot(x,y,'gs')

#for xy in zip(x,y):
#    plt.annotate('(%s)' % mag, xy=xy, textcoords='data') 

#plt.xlim([min(xp),max(xp)/3+min(xp)])
plt.axis('equal')
#plt.plot(x,y)
f4.canvas.draw()

he_min=100;
for i in range(len(x)-1):
    he_min=min(math.sqrt((x[i+1]-x[i])**2+(y[i+1]-y[i])**2),he_min)
print he_min

display(VBox([f4.canvas]))

In [ ]:
he_min=100;
for i in range(len(x)-1):
    he_min=min(math.sqrt((x[i+1]-x[i])**2+(y[i+1]-y[i])**2),he_min)
print he_min